In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
PATH = 'important_backlog/'

In [ ]:
with open(os.path.join(PATH, 'activities_clusters.pickle'), 'rb') as f:
    activities_clusters = pickle.load(f)

In [16]:
with open(os.path.join(PATH, 'books_clusters.pickle'), 'rb') as f:
    books_clusters = pickle.load(f)

In [18]:
##########
# id_book_dict = dict()
# for _, books_in_cluster in books_clusters.items():
#     for book in books_in_cluster:
#         id_book_dict[book['book_id']] = book['title']

In [20]:
# with open('important_backlog/idbook_booktitle.pickle', 'wb') as f:
#     pickle.dump(id_book_dict, f)

In [ ]:
with open(os.path.join(PATH, 'courses_clusters.pickle'), 'rb') as f:
    courses_clusters = pickle.load(f)

In [11]:
with open(os.path.join(PATH, 'readers_clusters.pickle'), 'rb') as f:
    readers_clusters = pickle.load(f)

In [12]:
list(readers_clusters.keys())[:10]

[163, 164, 165, 168, 170, 171, 173, 174, 176, 177]

In [ ]:
with open(os.path.join(PATH, 'pupils_clusters.pickle'), 'rb') as f:
    pupils_clusters = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'average_pupils_ratios.pickle'), 'rb') as f:
    average_pupils_ratios = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'average_readers_ratios.pickle'), 'rb') as f:
    average_readers_ratios = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'last_book_predictor.pickle'), 'rb') as f:
    last_book_predictor = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'last_course_predictor.pickle'), 'rb') as f:
    last_course_predictor = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'clusters_mapper.pickle'), 'rb') as f:
    clusters_mapper = pickle.load(f)

In [14]:
with open(os.path.join(PATH, 'collaborative_predictor_NOT_FULL.pickle'), 'rb') as f: #to create
    collaborative_predictor = pickle.load(f)

In [ ]:
with open(os.path.join(PATH, 'readers_books_set.pickle'), 'rb') as f: #to create
    readers_books_set = pickle.load(f)

In [ ]:
users = list(readers_clusters.keys())
pupils = list(pupils_clusters.keys())

In [ ]:
from datetime import datetime
from datetime import timedelta

In [ ]:
# skip SVD
# skip age constraints (needed to be adjusted in databases)
def predict(user_id):
    if user_id in users:
        user_genres = readers_clusters[user_id]['book_ratios']
        dominant_cluster = str()
        max_ratio = int()
        for key, value in user_genres.items():
            if value > max_ratio:
                max_ratio = value
                dominant_cluster = key
        user_age = readers_clusters[user_id]['age']
        
        # top 10 last item
        last_item = readers_clusters[user_id]['last_book']
        last_item_top_10 = set(last_book_predictor[last_item])
        
        # top 10 collaborative
        the_closest = collaborative_predictor[user_id]
        user_set = readers_books_set[user_id]
        the_closest_set = readers_books_set[the_closest][0]
        collaborative_top_10 = the_closest_set - user_set
        
        top_5_books = []
        both_predict = last_item_top_10 & collaborative_top_10
        if both_predict:
            top_5_books = list(both_predict)[:5]
            
        diff = 5 - len(top_5_books)
        if diff > 0:
            top_5_books = top_5_books + last_item_top_10[:diff]
            
        # courses
        recommended_course = ''
        course_cluster, event_cluster = clusters_mapper[dominant_cluster]
        if course_cluster:
            recommended_course = courses_clusters[course_cluster][0]['title'] # assume storaging popularity sorted
            
        # events
        recommended_event = ''
        today = datetime.today()
        time_related_events = activities_clusters['time_related']
        for time_rel in time_related_events:
            delta = (time_rel['event_date'] - today).days
            if delta <= 7: #a week before
                recommended_event = time_rel['title']
                break
                
        if not recommended_event:
            if event_cluster:
                recommended_event = activities_clusters[event_cluster][0]['title']
                
        return top_5_books, recommended_course, recommended_event
    
    else:
        user_courses = pupils_clusters[user_id]['course_ratios']
        dominant_cluster = str()
        max_ratio = int()
        for key, value in user_courses.items():
            if value > max_ratio:
                max_ratio = value
                dominant_cluster = key
#         user_age = pupils_clusters[user_id]['age_cat']
        
        # top 10 last item
        last_item = pupils_clusters[user_id]['last_course']
        last_item_top_10 = set(last_course_predictor[last_item])
        
        top_5_courses = last_item_top_10[:5]
        
        # books
        recommended_book = ''
        book_cluster, event_cluster = clusters_mapper[dominant_cluster]
        if book_cluster:
            recommended_book = books_clusters[book_cluster][0]['title'] # assume storaging popularity sorted
            
        # events
        recommended_event = ''
        today = datetime.today()
        time_related_events = activities_clusters['time_related']
        for time_rel in time_related_events:
            delta = (time_rel['event_date'] - today).days
            if delta <= 7: #a week before
                recommended_event = time_rel['title']
                break
                
        if not recommended_event:
            if event_cluster:
                recommended_event = books_clusters[event_cluster][0]['title']
                
        return top_5_courses, recommended_book, recommended_event

Алгоритм работы (упрощённая версия)

получить на вход id юзера
по словарю восстановить профиль юзера - наибольшую ratio кластера (доминирующий кластер) и возраст # другие фичи не берутся в расчёт
понять, он ридер или ученик?

Если он ридер:
    # Предсказываем топ книг
    получить топ 10 рекомендаций last_item предиктора
    получить топ 10 рекомендаций SVD предиктора
    получить топ 10 рекомендаций коллаборативной фильтрации
    
    создать пустой список топ 5 рекомендаций
    попарно найти пересечения всех предикторов
    Если пересечения есть:
        добавить их в топ 5
    
    Найти разницу между 5 и длиной топ 5
    Если > 0:
        добавить в топ 5 разницу из last item
    
    # кластер не используется для сортировки
    
    # Предсказываем топ кружков
    рекомендуемый кружок = ''
    по доминирующему кластеру ищем в мэппинге смежный кластер с кружками
    Если кластер есть: # добавить в файл по кластерам кружков фичу о популярности кружка
        индекс = 0
        До тех пор пока возраст ридера не будет соответствовать возрастным ограничениям кружка:
            индекс += 1
        получаем кружок для рекомендации
    
    # Предсказываем топ мероприятий
    рекомендуемое мероприятие = ''
    получаем текущую дату
    для time related мероприятий находим дельту между датой мероприятия и текущей датой
    Если дата < 7:
        Если возраст соответствует:
            получаем мероприятие для рекомендации
    
    по доминирующему кластеру ищем в мэппинге смежный кластер с мероприятиями
    Если кластер есть: # нужна популярность
        индекс = 0
        До тех пор пока возраст ридера не будет соответствовать возрастным ограничениям мероприятия:
            индекс += 1
        получаем мероприятие для рекомендации
    
    возвращаем топ-5, кружок, мероприятие
    
Если он ученик:
    # Предсказываем топ кружков
    получить топ 10 рекомендаций last_item предиктора
    топ5 для ученика = топ10[:5]
    
    # Предсказываем топ книг
    рекомендуемая книга = ''
    по доминирующему кластеру ищем в мэппинге смежный кластер с книгами
    Если кластер есть: # добавить в файл по кластерам кружков фичу о популярности книг
        берём первую по популярности книгу
    
    # Предсказываем топ мероприятий
    рекомендуемое мероприятие = ''
    получаем текущую дату
    для time related мероприятий находим дельту между датой мероприятия и текущей датой
    Если дата < 7:
        Если возраст соответствует:
            получаем мероприятие для рекомендации
    
    по доминирующему кластеру ищем в мэппинге смежный кластер с мероприятиями
    Если кластер есть: # нужна популярность
        индекс = 0
        До тех пор пока возраст ридера не будет соответствовать возрастным ограничениям мероприятия:
            индекс += 1
        получаем мероприятие для рекомендации
    
    возвращаем топ-5, кружок, мероприятие